In [13]:
import pandas as pd
import numpy as np
from headers import *
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from utils.custom_train_test_split import *

In [14]:
df = joinDfs(['../data/combined_inner_ticker.csv', '../data/extra-data/combined_inner_ticker.csv'])
X = df[ratioKeys + relativeRatioKeys]
X = sm.add_constant(X)

In [15]:
models = {}
importantDict = {}

In [16]:
for yLabel in yAlpha:
    y = df[yLabel]
    results = sm.OLS(y, X).fit()
    print(results.summary())
    models[yLabel] = results

                            OLS Regression Results                            
Dep. Variable:             alpha1Year   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     21.62
Date:                Thu, 12 Jun 2025   Prob (F-statistic):           3.97e-71
Time:                        21:11:43   Log-Likelihood:                -18679.
No. Observations:               41274   AIC:                         3.740e+04
Df Residuals:                   41255   BIC:                         3.756e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const           

In [17]:
for yLabel in yAlpha:
    model = models[yLabel]
    pvalues = model.pvalues
    important = pvalues.where(lambda x: x < 0.05).dropna()
    print(f'For {yLabel} significant (p<=0.05) are:')
    print(important)
    print('-----------------')
    for key in important.index:
        if key not in importantDict:
            importantDict[key] = 0
        importantDict[key] += 1

For alpha1Year significant (p<=0.05) are:
const                  2.079694e-66
evToEbit               8.151024e-03
evToGrossProfit        5.753751e-09
priceToGrossProfit     2.514456e-04
psRatio_relative       6.421470e-05
peRatio_relative       1.517526e-13
evToEbitda_relative    1.849358e-03
evToEbit_relative      3.543533e-07
evToSales_relative     1.004170e-02
dtype: float64
-----------------
For adjustedAlpha1Year significant (p<=0.05) are:
const                   8.779459e-79
psRatio                 5.143236e-03
evToEbit                9.450927e-03
evToGrossProfit         3.625845e-07
priceToGrossProfit      2.716639e-03
psRatio_relative        7.865519e-05
peRatio_relative        1.429611e-11
priceToBook_relative    3.948397e-02
evToEbitda_relative     6.441535e-03
evToEbit_relative       1.034400e-06
evToSales_relative      5.232810e-03
dtype: float64
-----------------
For equalAlpha1Year significant (p<=0.05) are:
const                           4.402594e-25
evToEbit           

How many times each column was classified as significant

In [20]:
importantDict = dict(sorted(importantDict.items(), key=lambda item: item[1], reverse=True))
# print(importantDict)
# for key, value in importantDict.items():
#     print(f'{key} was important variable {value} times')
for yLabel in ratioKeys + relativeRatioKeys:    
    if yLabel not in importantDict.keys():
        print(yLabel)

peRatio
evToEbitda
priceToFreeCashFlow
evToSales
priceToGrossProfit_relative


Linear predictions

In [19]:
for yLabel in yAlpha:
    y = df[yLabel]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)
    results = sm.OLS(y_train, X_train).fit()
    model = results.model
    res = results.get_prediction(X_test).predicted
    print(f'R2 score for {yLabel} is {r2_score(y_test, res)}')

R2 score for alpha1Year is 0.008857446629790489
R2 score for adjustedAlpha1Year is 0.009071845341300122
R2 score for equalAlpha1Year is 0.009633059938762845
R2 score for equalAdjustedAlpha1Year is 0.009899230411672622
R2 score for alpha2Year is 0.004580910940286564
R2 score for adjustedAlpha2Year is 0.0055911183109749185
R2 score for equalAlpha2Year is 0.006007367400849173
R2 score for equalAdjustedAlpha2Year is 0.006910365311798228


R2 is basically 0 everywhere that means
## Linear models are not reliable regressor in this case
Will use SVM as classificator 
** See svm_predict.py, use_svm_models.py **